In [1]:
## This is a demo for atlas
## Author : Avadesh Meduri
## Date : 06/04/2022
%load_ext autoreload
%autoreload 2

import time
import numpy as np
from mpc.abstract_cyclic_gen1 import AbstractGaitGen
from robot_properties_atlas.config import AtlasConfig
from py_biconvex_mpc.ik_utils.abstract_gait_generator import AbstractGaitGenerator

from matplotlib import pyplot as plt

import pinocchio as pin

import numpy as np
from motions.weight_abstract import BiconvexMotionParams
from robot_properties_solo.config import Solo12Config

<frozen importlib._bootstrap>:219: RuntimeWarning: to-Python converter for boost::shared_ptr<hpp::fcl::CollisionGeometry> already registered; second conversion method ignored.


37 37


In [2]:
robot = AtlasConfig.buildRobotWrapper()
rmodel = robot.model
rdata = robot.data
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=False)
viz.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [3]:
## robot config and init
pin_robot = AtlasConfig.buildRobotWrapper()
urdf_path = AtlasConfig.urdf_path

eff_names = ["l_foot_lt", "l_foot_rt", "l_foot_lb", "l_foot_rb", "r_foot_lt", "r_foot_rt", "r_foot_lb", "r_foot_rb"]
hip_names = ["l_leg_hpz", "l_leg_hpz", "l_leg_hpz", "l_leg_hpz", "r_leg_hpz", "r_leg_hpz", "r_leg_hpz", "r_leg_hpz"]

# eff_names = ["l_leg_akx", "r_leg_akx"]
# hip_names = ["l_leg_hpz", "r_leg_hpz"]
n_eff = len(eff_names)

q0 = np.array(AtlasConfig.initial_configuration)
q0[0:2] = 0.0
q0[10] = np.pi/4.0
# q0[13] = np.pi/4.0

v0 = pin.utils.zero(pin_robot.model.nv)
x0 = np.concatenate([q0, pin.utils.zero(pin_robot.model.nv)])

v_des = np.array([0.0,0.0,0.0])
w_des = 0.0

plan_freq = 0.05 # sec
update_time = 0.0 # sec (time of lag)

In [4]:
gg = AbstractGaitGen(urdf_path, eff_names, hip_names, x0, plan_freq, q0)

IndexError: Index out of range

In [ ]:
#### Stand Still #########################################
still = BiconvexMotionParams("atlas", "Stand")

# Cnt
still.gait_period = 0.15
still.stance_percent = n_eff*[1.,]
still.gait_dt = 0.02
still.phase_offset = int(n_eff)*[0.0,]

# IK
still.state_wt = np.array([1e4, 1e4, 1e4] + [1e5] * 3 + [1e5] * (pin_robot.model.nv - 6) \
                         + [1e2] * 3 + [1e3] * 3 + [5.] *(pin_robot.model.nv - 6))

still.ctrl_wt = [0, 0, 1] + [1, 1, 1] + [5.0] *(rmodel.nv - 6)

still.swing_wt = [1e5, 2e5]
still.cent_wt = [5e+2, 5e+1]
still.step_ht = 0.
still.nom_ht = 1.12
still.reg_wt = [5e-2, 1e-5]

# Dyn
still.W_X =     np.array([5e+3, 5e+3, 1e+5, 1e-2, 1e-2, 1e1, 1e-2, 1e-2, 1e-2])
still.W_X_ter = 10.*np.array([1e3, 1e3, 1e+5, 1e+2, 1e+2, 2e3, 1e+2, 1e+2, 1e+2])
still.W_F = np.array(8*[1e1, 1e1,5e0])
still.rho = 1e4

still.ori_correction = [0.5, 0.5, 0.5]
still.gait_horizon = 1

# Gains
still.kp = 150.0
still.kd = 10.0

In [ ]:
gg.update_gait_params(still, 0)

In [ ]:
viz.viewer.jupyter_cell()

In [ ]:
sim_t = 0.0
sim_dt = 0.001
index = 0
pln_ctr = 0
q = q0
v = v0


for o in range(100):
    xs, us, f = gg.optimize(q, v, sim_t, v_des, w_des)
#     gg.plot(q, v)
    for ind in range(len(xs)):
        viz.display(xs[ind][:robot.model.nq])
        time.sleep(0.005)

    q = xs[int(plan_freq/sim_dt)-1][0:pin_robot.model.nq]
    v = xs[int(plan_freq/sim_dt)-1][pin_robot.model.nq:]
    sim_t += plan_freq

In [5]:
rmodel

Nb joints = 32 (nq=37,nv=36)
  Joint 0 universe: parent=0
  Joint 1 root_joint: parent=0
  Joint 2 back_bkz: parent=1
  Joint 3 back_bky: parent=2
  Joint 4 back_bkx: parent=3
  Joint 5 l_arm_shz: parent=4
  Joint 6 l_arm_shx: parent=5
  Joint 7 l_arm_ely: parent=6
  Joint 8 l_arm_elx: parent=7
  Joint 9 l_arm_wry: parent=8
  Joint 10 l_arm_wrx: parent=9
  Joint 11 l_arm_wry2: parent=10
  Joint 12 neck_ry: parent=4
  Joint 13 r_arm_shz: parent=4
  Joint 14 r_arm_shx: parent=13
  Joint 15 r_arm_ely: parent=14
  Joint 16 r_arm_elx: parent=15
  Joint 17 r_arm_wry: parent=16
  Joint 18 r_arm_wrx: parent=17
  Joint 19 r_arm_wry2: parent=18
  Joint 20 l_leg_hpz: parent=1
  Joint 21 l_leg_hpx: parent=20
  Joint 22 l_leg_hpy: parent=21
  Joint 23 l_leg_kny: parent=22
  Joint 24 l_leg_aky: parent=23
  Joint 25 l_leg_akx: parent=24
  Joint 26 r_leg_hpz: parent=1
  Joint 27 r_leg_hpx: parent=26
  Joint 28 r_leg_hpy: parent=27
  Joint 29 r_leg_kny: parent=28
  Joint 30 r_leg_aky: parent=29
  Joint

In [ ]:
q0.shape[0] - 7